# Tutorial dla Tweepy

## Podstawowe możliwosci

### Instalacja i uwierzytelnienie

Pobieramy najnowszą wersję.

Aby się uwierzytelnić używamy naszego tokenu z developer.twitter.com

Poniższy został przyznany w ramach grantu akademickiego, pozwala na pobranie 10 milionow Twittow miesięcznie i korzystać z wszystkich fukcjonalnosć Twittera API (przede wszystkim z wyszukiwania Twittow starszych niż tydzień).

In [1]:
academic_bearer = "AAAAAAAAAAAAAAAAAAAAADIEawEAAAAAxzzD4cQ2g8FGK2%2BkKz2%2FJvTnoMA%3D09uegYs5HrQvrsFkAEl3WwxhspBYFBIH3Vnykec79asqiUsSoA"

In [5]:
import tweepy 
import pandas as pd
import re

In [6]:
streaming = tweepy.StreamingClient(academic_bearer)

In [7]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 9.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 3.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=2cc4e4e1d72992b88a1a2a55a5b12e8b956526bdf88debb1b9e27f4f4516cc20
  Stored in directory: /Users/mwoj/Library/Caches/pip/wheels/51/c8/18/298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [248]:
from pyspark.sql import WriteSession
from pyspark.sql.types import StructType,StructField, StringType

spark = WriteSession \
        .builder \
        .appName('Test_threading') \
        .getOrCreate()
RDD = spark.sparkContext.emptyRDD()
schema = StructType([
  StructField('text', StringType(), True),
  StructField('tags', StringType(), True)
  ])
df = spark.createDataFrame(RDD,schema)

ImportError: ignored

In [21]:
import socket 
s = socket.socket()

    # Get local machine name : host and port
host = "localhost"
    # You'll want to make sure this port is being used elsewhere, otherwise you'll get an error
port = 60439
s.connect((host,port))
timeReceived = s.recv(1024)

ConnectionRefusedError: [Errno 61] Connection refused

### Wyszukiwanie Tweetow

W przypadku normalnego dostępu posługujemy się funkcją, ktora znajduje tweety z ostatniego tygodnia:

In [243]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

class IDPrinter(tweepy.StreamingClient):
    def __init__(self,academic_bearer,csocket):
      super().__init__(academic_bearer)
      self.client_socket = csocket

    def on_data(self, data):
      msg = json.loads( data )
      tags =[]
      #print(msg['data']['text'])
      tt = []
      text = msg['data']['text']
      global emoji_pattern
      tt.append(emoji_pattern.sub(r'', text))
      for rule in msg['matching_rules']:
        if(rule['id'] == "1616159160550178822"):
          tags.append('biden')
        else:
          tags.append('trump')
      #print(str("_".join(tags)))
      tt.append(str("_".join(tags)))
      self.client_socket\
            .send(str(text+"t_end")\
            .encode('utf-8'))
      global spark
      global schema
      global df
      newRow = spark.createDataFrame([tt], schema)





In [244]:
printer = IDPrinter("AAAAAAAAAAAAAAAAAAAAADIEawEAAAAAxzzD4cQ2g8FGK2%2BkKz2%2FJvTnoMA%3D09uegYs5HrQvrsFkAEl3WwxhspBYFBIH3Vnykec79asqiUsSoA")
printer.add_rules(tweepy.StreamRule("trump -is:reply -is:retweet -has:links lang:en"))
printer.add_rules(tweepy.StreamRule("biden -is:reply -is:retweet -has:links lang:en"))
printer.get_rules()

Response(data=[StreamRule(value='trump -is:reply -is:retweet -has:links lang:en', tag=None, id='1616159155525402629'), StreamRule(value='biden -is:reply -is:retweet -has:links lang:en', tag=None, id='1616159160550178822')], includes={}, errors=[], meta={'sent': '2023-01-19T21:15:32.110Z', 'result_count': 2})

In [245]:
printer.filter()

['Biden didn’t claim 600K on his Taxes? The DA, AG and the DOJ totally, tried to discredit Trump but, their not going after the real criminal? Biden is the Criminal. He Committed Treason and more. When is the DOJ going to do their Job? The FBI and DOJ has dbl standards.', 'trump_biden']



['President Joseph Biden and his Government are Euthanizing Retarded adults and Homosexuals over age 55 who are receiving SSDI payments from Social security. Loaded with cancer by SAC Norad and Nasa broadcasts of high UVA and Microwaves. and Stealing money. United Nations aware.', 'biden']



["I am dancing I'm my mine Alec Baldwin is getting charged with 2nd degree murder I can't wait til he gets to Folsom he should also be put in general pop I hope he does his Trump parady to they will like that and they'll love him with a mop on his head", 'trump']



["Hi, Mr. Clyburns...I'm running for 2024 and I look for to challenging Mr. Biden, my goal is to replace him in 2024 giving American back the feelin

KeyboardInterrupt: ignored

In [247]:
df.show()

+--------------------+-----------+
|                text|       tags|
+--------------------+-----------+
|Biden didn’t clai...|trump_biden|
|President Joseph ...|      biden|
|I am dancing I'm ...|      trump|
|Hi, Mr. Clyburns....|      biden|
+--------------------+-----------+



In [81]:
from tweepy.streaming import StreamingClient
import json

class TweetsListener(StreamingClient):
  # tweet object listens for the tweets
  def __init__(self, csocket):
    self.client_socket = csocket
  def on_data(self, data):
    try:  
      msg = json.loads( data )
      print("new message")
      # if tweet is longer than 140 characters
      if "extended_tweet" in msg:
        # add at the end of each tweet "t_end" 
        #self.client_socket\
         #   .send(str(msg['extended_tweet']['full_text']+"t_end")\
          #  .encode('utf-8'))         
        print(msg['extended_tweet']['full_text'])
      else:
        # add at the end of each tweet "t_end" 
        #self.client_socket\
         #   .send(str(msg['text']+"t_end")\
          #  .encode('utf-8'))
        print(msg['text'])
      return True
    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True
  def on_error(self, status):
    print(status)
    return True

In [82]:
printer = TweetsListener(academic_bearer)
printer.add_rules(tweepy.StreamRule("musk -is:reply -has:links lang:en"))
printer.filter()

AttributeError: ignored

In [94]:
printer.delete_rules(['1616155777885929499','1616154119747534852'])
printer.get_rules()

Response(data=None, includes={}, errors=[], meta={'sent': '2023-01-19T19:40:41.584Z', 'result_count': 0})

In [66]:
query = 'musk -is:reply -has:links lang:en'
tweet_fields = ['created_at','entities']
max_result = 10
entities=[]
tweets = client.search_recent_tweets(query=query, tweet_fields=tweet_fields, max_results=max_result)

for tweet in tweets.data:
    #print(tweet.text)
    #print(tweet.created_at)
    if (tweet.entities !=None):
      entities.append(tweet.entities)
    #print("\n\n")



In [67]:
#entities = list(filter(None, entities))
if (tweet.entities !=None):
  for ent in entities:
    if(ent.get('annotations') != None):
      for ann in ent.get('annotations'):
        print(ann.get('normalized_text'))



Elon Musk
Elon Musk
Elon Musk
Elon Musk
Jefferey Epstein
Epstein
Elo
Musk
MAGA
Musk
Elon Musk
Musk
Davos
McCarthy
Trump


Mając dostęp do grantu akademickiego możemy brać tweety starsze niż tydzień korzystając z funkcji ```search_all_tweets```, maksymalna liczba zwracanych tweetow to 500

In [6]:
query = 'from:juliaioffe -is:retweet'
tweet_fields = ['created_at']
max_result = 250

tweets = client.search_all_tweets(query=query, tweet_fields=tweet_fields, max_results=max_result)

for tweet in tweets.data:
    print(tweet.text)
    print(tweet.created_at)
    print("\n\n")

Hey, D.C. area folks! Come hear me and the lovely @MarshKatherine talk about her INCREDIBLE new Y.A. book about the Ukrainian famine, "The Lost Year." I couldn't put it down. 

Connecticut Avenue @PoliticsProse at 7pm! https://t.co/DelluIHSDr
2023-01-17 17:29:48+00:00



@Swift818 He can be anything you want him to be!
2023-01-15 18:41:32+00:00



Stay classy, Dima. https://t.co/YO7zAMOqVq
2023-01-15 18:34:55+00:00



🤦🏻‍♀️
https://t.co/ZgwIj1teIR https://t.co/wcH1NRT0DB
2023-01-14 16:46:38+00:00



🥺 https://t.co/2OLcPojwtH
2023-01-14 16:40:45+00:00



Absolutely horrific https://t.co/rwcKbp8jhP
2023-01-14 16:39:36+00:00



Of course. 

https://t.co/g8KYdXv1it
2023-01-12 17:37:10+00:00



@mollyesque 💔
2023-01-11 17:58:07+00:00



At the Pentagon, where the leadership had been preparing for hearings on Afghanistan and Ukraine, suddenly people started wondering if they should also prepare to explain how much D.E.I. training cost in comparison to a Bradley.

My latest.
https://t.co/DXeO

### Zapis wielu tweetow do pliku

In [ ]:
query = 'from:juliaioffe -is:retweet'
tweet_fields = ['created_at']
max_result = 500
number_of_wanted_tweets = 1000

file_name = "many_tweets.tsv"
with open(file_name, 'a+') as filehandle:
    for tweet in tweepy.Paginator(client.search_all_tweets, query=query, tweet_fields=tweet_fields, max_results=max_result).flatten(limit=number_of_wanted_tweets):
        filehandle.write(f"{tweet.text}\t{tweet.created_at}\n")

### Użycie query z rożnymi znacznikami

Lista znajduje się [tutaj](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)

Poniżej kilka przykładow:



In [ ]:
query = '#Ukraine lang:en place_country:RU'
tweet_fields = ['created_at']
max_result = 50

tweets = client.search_all_tweets(query=query, tweet_fields=tweet_fields, max_results=max_result)

for tweet in tweets.data:
    print(tweet.text)
    print(tweet.created_at)
    print("\n\n")

Good luck #Ukraine #UKR #EUROVISION
2022-05-10 19:33:09+00:00



Our discord server is being updated. Log in to the server using the link in the bio. #Crypto #Elon_Musk #ETH #NFTs #Discord #DINO #dinosaur #pixelart #Ukraine #NFTartist #matic #PolygonNFT #dinotravels #dinosaur https://t.co/OWuw2PmrsZ
2022-05-10 19:04:28+00:00



Flowers in Tushino
#moscow #tushino #peace #flowers #spring #relax #poland #ukraine #тушино #сзао #москва #цветы #мир #весна #украина #парк #отдых https://t.co/3o2TP5Ts7a
2022-05-01 14:54:34+00:00



Missile launches over Belgorod https://t.co/ipfMp4uyFU via @GirkinGirkin #Ukraine https://t.co/MPqwIIcFpl
2022-04-28 14:51:27+00:00



5 PUTIN Rumors Rated: From Autism to Roid Rage to Parkinson’s 🤔⬇️⬇️⬇️

See our analysis at https://t.co/ZqosJezDzB 👁‼️

#Putin #Ukraine #LifeLineMedia #LifeLineNews #UncensoredNews https://t.co/BcSezU3z08
2022-04-26 15:44:01+00:00



Russian Foreign Minister Lavrov threatens with real probability of beginning of the Third World War h

### Oczyszczania tweetow

Informacje o wspomnianych użytkownikach oraz o retweetach znajduje się także w Twitter fields. 

In [ ]:
import re

In [ ]:
RETWEETED_REGEX = r"^RT @([^ ]+):"
USER_REGEX = r"@([^ ]+)"
URL_REGEX = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"


In [ ]:
def clean_tweet(tweet):
    is_retweet = False if re.match(RETWEETED_REGEX, tweet) is None else True
    di = {"is_retweet": is_retweet,
          "retweeted_user": re.findall(RETWEETED_REGEX, tweet)[0] if is_retweet else "",
          "links": re.findall(URL_REGEX, tweet)}
    tweet = re.sub(RETWEETED_REGEX, "", tweet)
    di["mentioned_users"] = re.findall(USER_REGEX, tweet)
    tweet = re.sub(USER_REGEX, "", tweet)
    tweet = re.sub(URL_REGEX, "", tweet)
    di["cleaned_tweet"] = tweet.strip()
    return di

### Użytkownicy

Jak znaleźć użytkownikow danych tweetow

In [ ]:
query = 'Ukraina lang:pl place_country:PL'
tweet_fields = ['created_at']
max_result = 40
expansion = "author_id"

tweets = client.search_all_tweets(query=query, tweet_fields=tweet_fields, expansions=expansion, max_results=max_result)

users = []

for tweet in tweets.data:
    tweet_dict = clean_tweet(tweet.text)
    #print(tweet.author_id)
    user = client.get_user(id=tweet.author_id, user_fields=["username"])
    print(user.data.name)
    print(user.data.username)  #Username jest najważniejsze
    print(tweet_dict["cleaned_tweet"])
    users.append(user.data.username)
    print("\n\n")

Maja Magdalena Piesik 🇵🇱
majxcia
Bo to jest POLSKA, a nie Ukraina. Tu mamy swój system oświaty. Nawet, jeśli wymaga reformy. Także, nie pogrążaj się.



Grupa MTP
GrupaMtp
💬  Wojna w Ukrainie ma globalny zasięg. Dzięki smartfonom możemy pokazać te zbrodnie całemu światu. Robimy wszystko, co możemy, aby stać ramię w ramię z Ukrainą i aby to, co dzieje się w Europie Wschodniej nie zdestabilizowało świata globalnie.



Jacob
Jacob3kxa
Juleczki Życie wam nie miłe? To jest polska osoba i to jest polska #ukraina #Polska #Toruń #solidarność #skandal #kopernik #pomnik #freekopernik



Anna Jasiulewicz
JasiulewiczAnna
Jesteś śmieszny macron Ukraina jak najszybciej powinna być włączona we wspólnotę europejską bo dzięki Ich heroicznej walce o swój Ukochany Kraj walczą też o naszą wolność żeby i u nas ta szarańcza się nie zaległa Ukraina musi zostać potraktowana wyjątkowo i jak najszybciej być UE



Jacek Hecht
Jacek_Hecht
Trwa blokada ukraińskich portów na Morzu Czarnym, przez które 🇺🇦 realizował

Jesli chcemy wypisać tweety konkretnych użytkownikow

In [ ]:
users_to_check_list = ["AgnieszkaO2GA",
                          "BeataDbrowska9",
                          "Albert301271",
                          "RenaTa99609630",
                          "DemonDakki",
                          "PoPrawnik2",
                          "MarcJachacz"]

users_to_check_ids = [client.get_user(username=user).data.id for user in users_to_check_list]

In [ ]:
print(users_to_check_ids)

[1329885033063133189, 1487034661461233666, 32827670, 1260931245128572932, 958746678462382081, 976173218309509120, 818951639537160193]


In [ ]:
tweet_fields = ['created_at']
max_result = 10
for user_id, user_name in zip(users_to_check_ids, users_to_check_list):
  print(user_name)
  tweets = client.get_users_tweets(user_id, max_results=max_result, tweet_fields=tweet_fields)
  for tweet in tweets.data:
    print(tweet.text)
    print(tweet.created_at)
  print("END OF THIS USER \n\n\n")

AgnieszkaO2GA
Kuźwa ma się czym chwalić https://t.co/1qiR2S8auO
2022-05-11 12:36:33+00:00
RT @6_wilku: @PremierRP @MorawieckiM @harari_yuval @ImpactCEE Jak można spotykać się z takim szurem i do tego tym się chwalić?
https://t.co…
2022-05-11 12:35:53+00:00
RT @KotSzary: @PremierRP @MorawieckiM @harari_yuval @ImpactCEE Wiesz, że ten drań chce się pozbyć większości ludzi na świecie?
Dał ci instr…
2022-05-11 12:35:40+00:00
@Czarny58659094 😂
2022-05-11 11:21:12+00:00
@ParaJozef 😂
2022-05-11 11:11:31+00:00
@Radek030682 Jestem przekonana 😂
2022-05-11 11:09:44+00:00
👏👏👏👏👏👏 https://t.co/3IP4PdY6kL
2022-05-11 11:03:21+00:00
@JacekNieuk 😂 😂 😂 Wariot
2022-05-11 11:01:10+00:00
👇 https://t.co/IjNdjL4I6d
2022-05-11 11:00:40+00:00
@ukasz19888 😂
2022-05-11 10:04:27+00:00
END OF THIS USER 



BeataDbrowska9
RT @KotSzary: @PremierRP @MorawieckiM @harari_yuval @ImpactCEE Wiesz, że ten drań chce się pozbyć większości ludzi na świecie?
Dał ci instr…
2022-05-11 13:02:37+00:00
RT @waski96: @MariuszJagora @Jb

### Przykładowa klasa do sprawdzania podstawowych parametrow związanych z użytkownikiem:

In [ ]:
!pip install matplotlib
!pip install pandas
!pip install wordcloud


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def create_word_cloud(lista):
  comment_words = ''
  stopwords = set(STOPWORDS)
  print(lista)
  docs = lista["cleaned_tweet"].apply(nlp)
  for token in docs[0]:
    print(token)
  for item in lista:
      
      docs = item["cleaned_tweet"].apply(nlp)
      for token in docs[0]:
        if (token.dep_ == "amod" or token.dep_ == "nmod"):
          comment_words += " ".join(token.text.lower())+" "
  
  wordcloud = WordCloud(width = 800, height = 800,
                  background_color ='white',
                  stopwords = stopwords,
                  min_font_size = 10).generate(comment_words)
  
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  
  plt.show()

In [ ]:
from datetime import datetime
from datetime import timedelta

def get_old_data(last_days_num):
       now = datetime.now()
       then = now - timedelta(days=last_days_num)
       month = str(then.month).zfill(2)
       day = str(then.day).zfill(2)
       start_date = f"{then.year}-{month}-{day}T00:00:00Z"
       return start_date

In [ ]:
class UserStats(object):
    def __init__(self, user):
        self.client = client
        self.username = user
        user = self.client.get_user(username=user, user_fields=["created_at", "public_metrics", "verified"])
        self.id = user.data.id
        self.created = user.data.created_at
        self.followers_number = user.data.public_metrics["followers_count"]
        self.followed_number = user.data.public_metrics["following_count"]
        self.tweet_number = user.data.public_metrics["tweet_count"]
        self.if_verified = user.data.verified

    def get_tweets(self, previous_days=30):
        self.previous_days = previous_days
        old_data = get_old_data(previous_days)
        query = f"from:{self.username}"
        self.last_tweets = self.client.search_all_tweets(query, start_time=old_data, max_results=300, tweet_fields=["public_metrics"])
        self.processed_tweets = [clean_tweet(tweet.text) for tweet in self.last_tweets.data]

    def calculate_popularity(self):
        self.popularity_score = sum([tt.public_metrics["retweet_count"] for tt in self.last_tweets.data]) + \
                                sum([tt.public_metrics["like_count"] for tt in self.last_tweets.data])
        self.reach_score = self.followers_number - self.followed_number

    def get_followers(self, max_results=10):
        followers = self.client.get_users_followers(self.id, max_results=max_results)
        return [follower.username for follower in followers.data]

    def print_basic_stats(self):
        print(f"Popularity score: {self.popularity_score}")
        print(f"Reach score: {self.reach_score}")


In [ ]:
    kristen = UserStats(user="netsirk2c")
    kristen.get_tweets(previous_days=3600)
    kristen.calculate_popularity()
    kristen.print_basic_stats()
    list_of_followers = kristen.get_followers()
    for follower in list_of_followers:
        print(follower)

Popularity score: 1128589
Reach score: -161
KellyMaeGosnell
HurriCAIN21
_brooklyn10
Zayswag297
RachitK48861689
ijessicahere
ClaudeAbraham6
DonnaBerbal
MaireadMcMahon
HoeyB4


In [ ]:
create_word_cloud(kristen.processed_tweets)

[{'is_retweet': True, 'retweeted_user': '1DOPEKEV', 'links': [('', '/l8urwW0jyU')], 'mentioned_users': [], 'cleaned_tweet': 'me whenever i’m on a date bowling'}, {'is_retweet': True, 'retweeted_user': 'evenstarsss', 'links': [], 'mentioned_users': [], 'cleaned_tweet': 'anthony “i am a gentleman” (whore) bridgerton'}, {'is_retweet': False, 'retweeted_user': '', 'links': [('', '/cxBK2xBqkJ')], 'mentioned_users': [], 'cleaned_tweet': 'Apartment owners!'}, {'is_retweet': False, 'retweeted_user': '', 'links': [('', '/86G4Y9G2ES')], 'mentioned_users': [], 'cleaned_tweet': 'A live look at my life in Poland'}, {'is_retweet': False, 'retweeted_user': '', 'links': [], 'mentioned_users': ['KaleGuts'], 'cleaned_tweet': 'What happened 😭😭😂'}, {'is_retweet': False, 'retweeted_user': '', 'links': [], 'mentioned_users': [], 'cleaned_tweet': 'WE ARE BUYING AN APARTMENT'}, {'is_retweet': False, 'retweeted_user': '', 'links': [('', '/cgbEDWtWao')], 'mentioned_users': [], 'cleaned_tweet': 'This dude is abs

TypeError: ignored

### Zadania

1. Sprobowac pisac własne zapytania (query) korzystając z instrukcji na stronie dokumentacji
2. Moża poszukać roznego rodzaju propagandystow, stworzyć dla nich wordcloud, sprawdzić ich followersow itd. 
3. Stworzyć własną metrykę popularnosci np. srednia ilosc retweetow na dzień albo na pojedynczego tweeta - mozna wykorzystac te klasę

In [ ]:
query = 'from:cjshayshay -is:retweet'
tweet_fields = ['created_at']
max_result = 250

tweets = client.search_all_tweets(query=query, tweet_fields=tweet_fields, max_results=max_result)

for tweet in tweets.data:
    print(tweet.text)
    print(tweet.created_at)
    print("\n\n")

@japanesementor Exactly
2022-05-11 02:09:26+00:00



When they reconstruct your hands
2022-05-10 18:37:33+00:00



The original smile face
2022-05-10 18:34:54+00:00



How loud is dark matter
2022-05-10 02:59:02+00:00



What's past 100%
2022-05-10 02:58:33+00:00



@KaleGuts Lol
2022-05-09 18:37:17+00:00



@KaleGuts Good use of dummy
2022-05-09 18:00:38+00:00



Death to teams / slack message reactions
2022-05-09 17:46:53+00:00



G(oogle) chat just (re?)dropped
2022-05-08 23:23:54+00:00



Chatty bot
2022-05-08 23:18:46+00:00



You close the uber app and the car disappears
2022-05-08 23:12:01+00:00



@retsoor From me (me) to you (the button)
2022-05-08 22:37:28+00:00



Futures lean line got me spinning
2022-05-08 22:25:09+00:00



@retsoor Its connected to me... When i touch it
2022-05-08 22:22:35+00:00



@retsoor A button you can push
2022-05-08 22:10:27+00:00



@DINOTHING Honorable mention to DND
2022-05-08 22:02:36+00:00



Word
2022-05-08 21:58:38+00:00



'
2022-05-08 21:5